In [ ]:
import random
import numpy as np

from qiskit import QuantumCircuit, execute, Aer, IBMQ
from qiskit.visualization import plot_histogram, plot_state_qsphere as plot_q
from qiskit.quantum_info import Statevector

import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
from IPython.display import clear_output

from qiskit.providers.aer.noise import NoiseModel

from itertools import product #this one I use to find the prob of 00 without sampling. 

from qiskit import Aer, IBMQ, execute, BasicAer

statevector_sim = Aer.get_backend('statevector_simulator')
qasm_sim = Aer.get_backend('qasm_simulator')

In [ ]:
def rand_numb(low_in,high_in):
    return np.random.uniform(low=low_in, high=np.nextafter(high_in, np.inf))

#just to keep the upper limit included in the random number generation. 

In [ ]:
unitary_param_1 = [0.21249364, 2.33465111, 1.93219486, 2.55074749, 3.67705258, 6.15636742,
 2.26177908, 5.76752756, 2.01463033, 3.33610176]
unitary_param_2 = [3.80940494, 5.70236131, 0.42946852, 3.85949314, 4.95013009, 4.76411749,
 4.31178296, 3.81034009, 1.91721159, 4.63532783]

In [ ]:
# First unitary
qc1 = QuantumCircuit(5)
for i in range(5):
  qc1.rx(unitary_param_1[i], i)

for i in range(5):
  qc1.ry(unitary_param_1[i+5], i)

for i in range(4):
  qc1.cx(i, i+1)

qc1.draw()

┌─────────────┐┌────────────┐                    
q_0: ┤ Rx(0.21249) ├┤ Ry(6.1564) ├──■─────────────────
     └┬────────────┤├────────────┤┌─┴─┐               
q_1: ─┤ Rx(2.3347) ├┤ Ry(2.2618) ├┤ X ├──■────────────
      ├────────────┤├────────────┤└───┘┌─┴─┐          
q_2: ─┤ Rx(1.9322) ├┤ Ry(5.7675) ├─────┤ X ├──■───────
      ├────────────┤├────────────┤     └───┘┌─┴─┐     
q_3: ─┤ Rx(2.5507) ├┤ Ry(2.0146) ├──────────┤ X ├──■──
      ├────────────┤├────────────┤          └───┘┌─┴─┐
q_4: ─┤ Rx(3.6771) ├┤ Ry(3.3361) ├───────────────┤ X ├
      └────────────┘└────────────┘               └───┘

In [ ]:
# second unitary
qc2 = QuantumCircuit(5)
for i in range(5):
  qc2.rx(unitary_param_2[i], i)

for i in range(5):
  qc2.ry(unitary_param_2[i+5], i)

for i in range(4):
  qc2.cx(i, i+1)

qc2.draw()

┌────────────┐┌────────────┐                    
q_0: ─┤ Rx(3.8094) ├┤ Ry(4.7641) ├──■─────────────────
      ├────────────┤├────────────┤┌─┴─┐               
q_1: ─┤ Rx(5.7024) ├┤ Ry(4.3118) ├┤ X ├──■────────────
     ┌┴────────────┤├────────────┤└───┘┌─┴─┐          
q_2: ┤ Rx(0.42947) ├┤ Ry(3.8103) ├─────┤ X ├──■───────
     └┬────────────┤├────────────┤     └───┘┌─┴─┐     
q_3: ─┤ Rx(3.8595) ├┤ Ry(1.9172) ├──────────┤ X ├──■──
      ├────────────┤├────────────┤          └───┘┌─┴─┐
q_4: ─┤ Rx(4.9501) ├┤ Ry(4.6353) ├───────────────┤ X ├
      └────────────┘└────────────┘               └───┘

In [ ]:
#to understand if the circuit is correct!

circ = QuantumCircuit(8, 2)

circ.h(0)
circ.cx(0, 1)

      
circ.x(0)
#First controlled unitary
custom = qc1.to_gate().control(1)
circ.append(custom, [0, 2, 3, 4, 5, 6])
circ.x(0)
circ.barrier() 

      #second controlled unitary
custom = qc2.to_gate().control(1)
circ.append(custom, list(range(1, 7)))
circ.barrier()   

  # Applying the parametrized unitary

prover = [0, 2, 3, 7]
numlayer = 2
for j in range(numlayer):
  for i in range(len(prover)):
    circ.rx(params[j*2*len(prover) + i], prover[i])
    circ.ry(params[j*2*len(prover) + i + len(prover)], prover[i])

  for i in range(len(prover)-1):
    circ.cx(prover[i], prover[i+1])

  circ.barrier()

circ.cx(0, 1)
circ.h(0)

circ.barrier()

circ.draw()

┌───┐     ┌───┐              ┌───┐ ░                ░  ┌────────────┐»
q_0: ┤ H ├──■──┤ X ├──────■───────┤ X ├─░────────────────░──┤ Rx(5.7781) ├»
     └───┘┌─┴─┐└───┘      │       └───┘ ░                ░  └────────────┘»
q_1: ─────┤ X ├───────────┼─────────────░───────■────────░────────────────»
          └───┘     ┌─────┴──────┐      ░ ┌─────┴──────┐ ░ ┌─────────────┐»
q_2: ───────────────┤0           ├──────░─┤0           ├─░─┤ Rx(0.42153) ├»
                    │            │      ░ │            │ ░ └┬────────────┤»
q_3: ───────────────┤1           ├──────░─┤1           ├─░──┤ Rx(6.2523) ├»
                    │            │      ░ │            │ ░  └────────────┘»
q_4: ───────────────┤2 circuit-4 ├──────░─┤2 circuit-5 ├─░────────────────»
                    │            │      ░ │            │ ░                »
q_5: ───────────────┤3           ├──────░─┤3           ├─░────────────────»
                    │            │      ░ │            │ ░                »
q_6: ───────────────┤4           ├──────░─┤4           ├─░────────────────»
                    └────────────┘      ░ └────────────┘ ░  ┌────────────┐»
q_7: ───────────────────────────────────░────────────────░──┤ Rx(5.0129) ├»
                                        ░                ░  └────────────┘»
c: 2/═════════════════════════════════════════════════════════════════════»
                                                                          »
«     ┌────────────┐                ░  ┌──────────┐ ┌────────────┐          »
«q_0: ┤ Ry(3.0223) ├──■─────────────░──┤ Rx(6.01) ├─┤ Ry(3.9593) ├──■───────»
«     └────────────┘  │             ░  └──────────┘ └────────────┘  │       »
«q_1: ────────────────┼─────────────░───────────────────────────────┼───────»
«     ┌────────────┐┌─┴─┐           ░ ┌────────────┐┌────────────┐┌─┴─┐     »
«q_2: ┤ Ry(4.1658) ├┤ X ├──■────────░─┤ Rx(3.3681) ├┤ Ry(4.4567) ├┤ X ├──■──»
«     ├────────────┤└───┘┌─┴─┐      ░ ├────────────┤├────────────┤└───┘┌─┴─┐»
«q_3: ┤ Ry(1.9783) ├─────┤ X ├──■───░─┤ Rx(3.0083) ├┤ Ry(1.3194) ├─────┤ X ├»
«     └────────────┘     └───┘  │   ░ └────────────┘└────────────┘     └───┘»
«q_4: ──────────────────────────┼───░───────────────────────────────────────»
«                               │   ░                                       »
«q_5: ──────────────────────────┼───░───────────────────────────────────────»
«                               │   ░                                       »
«q_6: ──────────────────────────┼───░───────────────────────────────────────»
«     ┌────────────┐          ┌─┴─┐ ░ ┌───────────┐ ┌───────────┐           »
«q_7: ┤ Ry(4.0689) ├──────────┤ X ├─░─┤ Rx(4.971) ├─┤ Ry(2.646) ├───────────»
«     └────────────┘          └───┘ ░ └───────────┘ └───────────┘           »
«c: 2/══════════════════════════════════════════════════════════════════════»
«                                                                           »
«           ░      ┌───┐ ░ 
«q_0: ──────░───■──┤ H ├─░─
«           ░ ┌─┴─┐└───┘ ░ 
«q_1: ──────░─┤ X ├──────░─
«           ░ └───┘      ░ 
«q_2: ──────░────────────░─
«           ░            ░ 
«q_3: ──■───░────────────░─
«       │   ░            ░ 
«q_4: ──┼───░────────────░─
«       │   ░            ░ 
«q_5: ──┼───░────────────░─
«       │   ░            ░ 
«q_6: ──┼───░────────────░─
«     ┌─┴─┐ ░            ░ 
«q_7: ┤ X ├─░────────────░─
«     └───┘ ░            ░ 
«c: 2/═════════════════════
«

In [ ]:
def costf(param,numparam,numlayer):
    circ = QuantumCircuit(8, 2)

    circ.h(0)
    circ.cx(0, 1)

          
    circ.x(0)
    #First controlled unitary
    custom = qc1.to_gate().control(1)
    circ.append(custom, [0, 2, 3, 4, 5, 6])
    circ.x(0)
    circ.barrier() 

          #second controlled unitary
    custom = qc2.to_gate().control(1)
    circ.append(custom, list(range(1, 7)))
    circ.barrier()   

      # Applying the parametrized unitary

    prover = [0, 2, 3, 7]
    for j in range(numlayer):
      for i in range(len(prover)):
        circ.rx(param[j*2*len(prover) + i], prover[i])
        circ.ry(param[j*2*len(prover) + i + len(prover)], prover[i])

      for i in range(len(prover)-1):
        circ.cx(prover[i], prover[i+1])

      circ.barrier()

    circ.cx(0, 1)
    circ.h(0)

    circ.barrier()

    circ.draw()
        #circ.barrier()
        
    backend = BasicAer.get_backend('statevector_simulator')

    # Create a Quantum Program for execution
    job = execute(circ, backend)

    result = job.result()

    outputstate = result.get_statevector(circ, decimals=11)
    
    #We have generated the output state, and we measure the first two qubits in 0.
    ket=outputstate   

    res = [ele for ele in product(range(0,2), repeat=8)]
    for x in range(0, len(res)):
        res[x]=str(res[x][::-1]).replace(" ", "")[1::2]
        
    dic={}
    for i in range(0, len(ket)):
        dic.update({res[i]:np.real(ket[i]*np.conj(ket[i]))})
        
    
    res2 = [ele for ele in product(range(0, 2), repeat = 2)]
    for x in range(0, len(res2)):
        res2[x]=str(res2[x][::-1]).replace(" ", "")[1::2]

    dic2={}
    for ele in res2:
        dic2[ele] = 0
        
    for ele in dic:
        string= ele[:2]
        dic2[string]=dic2[string]+dic[ele]
    
    # cost
    olap = 1 - dic2['00']
    
    return(olap)


def costfp(param, numparam, index1,numlayer):
    opt_param_init = np.array([])
    for i in range(0,numparam):
        if i == index1:
            opt_param_init = np.append(opt_param_init, param[i]+np.pi/2)
        else:
            opt_param_init = np.append(opt_param_init, param[i])    
    
    return(costf(opt_param_init,numparam,numlayer))


def costfn(param, numparam, index1,numlayer):
    opt_param_init = np.array([])
    for i in range(0,numparam):
        if i == index1:
            opt_param_init = np.append(opt_param_init, param[i]-np.pi/2)
        else:
            opt_param_init = np.append(opt_param_init, param[i])    
    
    return(costf(opt_param_init,numparam,numlayer))

In [ ]:
# gradient descent
def gradient(fun1,fun2,fun3,param,numparam,numlayer,eta,eps_grad,eps_cost,count_num,gradcount):
    
    gcount=0
    count=0
    param1 = np.array([])
    costvalue = np.array([])
    dicc={}
    cost = fun1(param,numparam,numlayer)
    costvalue=np.append(costvalue, cost)
    
    num_parameters = numparam #change this if the ansatz changes
    
    for i in range(0,num_parameters):
        param1 = np.append(param1, param[i])
    while count<count_num and gcount<gradcount and cost > eps_cost:
        param2 = np.array([])
        param3 = np.array([])
        grad_vec=np.array([])
        
        # gradient value
        for i in range(0, num_parameters):
            grad_vec =np.append(grad_vec,(fun2(param1,num_parameters ,i,numlayer)-fun3(param1,num_parameters ,i,numlayer))/2)
        grad=np.linalg.norm(grad_vec)**2

        if grad<=eps_grad:
            gcount+=1

        param2 = param1-eta*grad_vec

        for i in range(0,num_parameters):
            param3 = np.append(param3, param2[i])

        param3 = param2 - eta*grad_vec
        # updating parameters
        if cost - fun1(param3,num_parameters,numlayer)>=eta*grad:
            eta=2*eta
            param1 = param3
        elif cost - fun1(param3,num_parameters,numlayer)<(eta/2)*grad:
            eta = eta/2
            param1=param2
        else:
            param1=param2

        cost = fun1(param1,num_parameters,numlayer)
        alpha_opt=param1
        

        dicc.update({count:np.array([cost, alpha_opt])})
        costvalue=np.append(costvalue, cost)
        print(count,cost)
        count+=1
        
        
    return(dicc)

In [ ]:

opt_param_init = np.array([])

numlayer= 5
numparam = 8*numlayer

for i in range(0,numparam):
    opt_param_init = np.append(opt_param_init, np.array([rand_numb(0,2*np.pi)]))

fun1 = costf
fun2 = costfp
fun3 = costfn

ans_test = gradient(fun1,fun2,fun3,opt_param_init,numparam,numlayer,0.5,1e-8,1e-8,300,50)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:47: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


0 0.6756460686790926
1 0.6218560444433077
2 0.5336998608524809
3 0.39752807583336225
4 0.32542994780656875
5 0.29844825574308675
6 0.24788932494729332
7 0.2310884856251456
8 0.22698788489889787
9 0.22066072219443977
10 0.21244498878852025
11 0.2061739659051075
12 0.21355884759720956
13 0.2460708215394929
14 0.21601012227565464
15 0.20335269912855136
16 0.20277218253812324
17 0.20251281111962371
18 0.2020993964063159
19 0.20180616659279382
20 0.2015326695996762
21 0.20132261509302296
22 0.20105212745225498
23 0.2011376753371742
24 0.20069736156682516
25 0.20040516069881587
26 0.20025370639350415
27 0.19998087754219263
28 0.19977697108381098
29 0.19961141582618358
30 0.19947364357683917
31 0.19935046395520128
32 0.1991681909818861
33 0.19917639075803417
34 0.19896134658520914
35 0.19879403958534836
36 0.19870838062852336
37 0.19855711368532314
38 0.19845195059911114
39 0.19836792587902152
40 0.19829855671668817
41 0.19823799680944254
42 0.19816376509963896
43 0.19824656763127513
44 0.198

In [ ]:
# calculate fidelity from cost
probab = 1-costf(ans_test[len(ans_test) -1][1], numparam, numlayer)
fid = (2*probab-1)**2
print(fid)

0.3712751799510587
